In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
pd.options.plotting.backend = "plotly"

import sys
sys.path.insert(0, 'driftpy/src/')
import driftpy
print(driftpy.__file__)

import numpy as np 
from sim.sim import DriftSim, load_hist_oracle

/Users/zanes/Documents/drift-sim-prod/driftpy/src/driftpy/__init__.py


In [ ]:
import os
import datetime
from sim.agents import * 
from programs.clearing_house.state import * 
from sim.events import OpenPositionEvent
from sim.helpers import random_walk_oracle, rand_heterosk_oracle, class_to_json
import pickle as cPickle

SIM_NAME = 'sim-results/sim-solhist'
# load_hist_oracle('LUNA-PERP', 'sim-solhist/oracle_prices.csv')

def make_ch(base_spread, strategies=''):
    oracle = Oracle(SIM_NAME+'/oracle_prices.csv')
    # oracle.to_csv(SIM_NAME+'/oracle_prices.csv')
    amm = AMM(
        oracle=oracle, 
        base_asset_reserve=int(367621.62052551797 * 1e13), 
        quote_asset_reserve=int(367621.62052551797 * 1e13),
        funding_period=60*60,
        peg_multiplier=int(oracle.prices[0]*1e3),
        base_spread = base_spread,
        strategies = strategies
    )
    market = Market(amm)
    
    if strategies=='':
        strategies = 'v1'
    ch_name = strategies+'_'+str(base_spread)
    
    fee_structure = FeeStructure(numerator=1, denominator=1000)
    clearing_house = ClearingHouse([market], fee_structure)
    clearing_house.name = ch_name

    return clearing_house


arb_agent1 = Noise(1, 0, 0, lookahead=0, size=10)
arb_funding1 = ArbFunding(1, 0, 1,)
agents = [arb_agent1, arb_funding1]

sim1 = DriftSim(SIM_NAME, clearing_house=make_ch(0), agents=agents)
sim2 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, ''), agents=agents)
sim2 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'PrePeg'), agents=agents)
sim3 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'InventorySkew'), agents=agents)
sim4 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'VolatilityScale'), agents=agents)
sim5 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'OracleRetreat'), agents=agents)
sim6 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'PreFreePeg_InventorySkew_OracleRetreat'),
                agents=agents)


sims = [sim1, sim2, sim3, sim4, sim5, sim6]
sims = [sim6]

for sim_i in sims:
    res = sim_i.run()
    print('finish sim')
    res_df = sim_i.to_df(save=True)
    print('finish res')
    print(res_df.columns)
    toplt = res_df[[
        'm0_mark_price', 'm0_oracle_price', 'm0_bid_price', 'm0_ask_price',
        # 'bid_price_before', 'ask_price_before',
        # 'user0.quote_asset_amount',
        # 'cumulative_funding_rate_long',
        'm0_peg_multiplier', 'm0_wouldbe_peg', 'm0_wouldbe_peg_cost',
        # 'm0_total_exchange_fees', 'm0_total_mm_fees',
        # 'm0_total_fee', 'm0_total_fee_minus_distributions',


        # 'base_asset_amount',
        # 'user0.base_asset_amount', 'user0.total_collateral', 'mark_std', 'last_spread',
        # 'total_fees', 
        # 'predicted_long_funding',
        # 'total_exchange_fees', 'total_mm_fees'
    ]].replace(0, np.nan)
    if 'u0_m0_base_asset_amount' in res_df.columns:

        toplt = pd.concat([toplt, res_df[['u0_m0_base_asset_amount']]],axis=1)
        toplt['u0_m0_base_asset_amount']/=1e13
        # toplt['user0.quote_asset_amount']/=1e6
    toplt['m0_peg_multiplier']/=1e3

    # toplt['base_asset_amount']/=1e13
    fig = toplt.plot(title=sim_i.clearing_house.name)
    fig.show()

running simulation for 27571 timesteps
running sim from timestamp 0 to 27571
skipping time step: 0 ... ch time: 3
skipping time step: 1 ... ch time: 3
skipping time step: 2 ... ch time: 3
NOW:  7
short 6.306659 LUNA-PERP @ 169.89627497427932 ( 169.8962577047 )
NOW:  8
short 46.673414 LUNA-PERP @ 169.89618515708017 ( 169.8960582323 )
NOW:  9
short 86.814363 LUNA-PERP @ 169.89587657915305 ( 169.8956404878 )
NOW:  10
short 5.879166 LUNA-PERP @ 168.28510164931714 ( 168.2850856409 )
NOW:  11
short 3.843988 LUNA-PERP @ 168.28512927874698 ( 168.2851188529 )
NOW:  12
short 1.809953 LUNA-PERP @ 168.28516837554864 ( 168.2851630204 )
NOW:  25
short 1.929008 LUNA-PERP @ 169.94241993601528 ( 169.9424144416 )
NOW:  26
short 42.712414 LUNA-PERP @ 169.94235266686147 ( 169.9422365071 )
NOW:  27
short 83.299161 LUNA-PERP @ 169.9420644208642 ( 169.9418378762 )
NOW:  28
short 12.17711 LUNA-PERP @ 169.16276699595235 ( 169.1627339288 )
NOW:  29
short 9.240398 LUNA-PERP @ 169.1120232128327 ( 169.1119980776 )

 23%|███████████████▌                                                    | 12634/55139 [00:12<00:42, 1003.09it/s]

budget k params 8.85009765625e-09 327262.80281489284 323327.65022650314 -2234.359283497472 173.513
280079215999405.56 -6.427209254744906 941.3824045760347
280079215999412.0 280079216000346.94
P, PK: 1.0 0.9999999999966619
budget k params 8.85009765625e-09 327262.80281489284 323327.65022650314 -2234.359283497472 173.513
280079215999405.56 -6.427209254744906 941.3824045760347
280079215999412.0 280079216000346.94
P, PK: 1.0 0.9999999999966619
budget k params 8.85009765625e-09 327262.80281489284 323327.65022650314 -2234.359283497472 173.513
280079215999405.56 -6.427209254744906 941.3824045760347
280079215999412.0 280079216000346.94
P, PK: 1.0 0.9999999999966619
budget k params 8.85009765625e-09 327262.80281489284 323327.65022650314 -2234.359283497472 173.513
280079215999405.56 -6.427209254744906 941.3824045760347
280079215999412.0 280079216000346.94
P, PK: 1.0 0.9999999999966619
budget k params 8.85009765625e-09 327262.80281489284 323327.65022650314 -2234.359283497472 173.513
2800792159994

100%|█████████████████████████████████████████████████████████████████████| 55139/55139 [00:56<00:00, 981.07it/s]


In [ ]:
(res_df[['m0_total_fee', 'm0_total_fee_minus_distributions']]).plot()

In [ ]:
(res_df['m0_quote_asset_reserve'].astype(int)/res_df['m0_base_asset_reserve'].astype(int)).plot()

In [ ]:
res_df[['m0_base_asset_reserve', 'm0_quote_asset_reserve', 'm0_sqrt_k',]].astype(int).plot()

In [ ]:
# ! echo 'hi'

In [ ]:
# ! cd protocol-v1/ && bash stress/run_local_sim.sh ../sim-results/sim-solhist/chPrepeg_1000.0

In [ ]:
# toplt['total_exchange_fees'].diff().plot()

# (res_df['user0.base_asset_amount']/1e13).plot()

# res_df['peg_multiplier'].plot()

In [ ]:
user = 'u1'
(res_df[[user+'_'+x for x in ['collateral',
       # 'u0_m0_upnl', 
         'total_collateral',
       'm0_ufunding',
       'm0_base_asset_amount',
         # 'user0.free_collateral',
       # 'user0.margin_ratio', 'user0.total_position_value'
       ]]]/1e6 - 10e6).plot()

In [ ]:
res_df[['m0_cumulative_funding_rate_long']].plot()

In [ ]:
entry_price =((res_df['u0_m0_quote_asset_amount']*1e7)/res_df['u0_m0_base_asset_amount'])
pd.concat([entry_price, res_df['m0_mark_price'],res_df['m0_ask_price']],axis=1).plot()

In [ ]:
# res_df[['repeg_to_oracle_cost', 'total_fee', 'total_exchange_fees', 'total_mm_fees',
#         'total_fee_minus_distributions']].plot()

In [ ]:
print(res_df.columns)
r = res_df[['m0_last_oracle_price_twap',
       'm0_last_mark_price_twap',
        'm0_last_bid_price_twap',
            'm0_last_ask_price_twap',
       'm0_last_mid_price_twap',
       ]]
# r = res_df[['oracle_price',
#        'mark_price',
#         'bid_price',
#             'ask_price',
#        ]]
# r['mid_price'] = (r['bid_price']+r['ask_price'])/2
r.plot()

In [ ]:
res_df[['m0_predicted_long_funding','m0_predicted_short_funding']].plot()